# 02 — RNN pooling fix (SmartWatch Gestures)

# Doel
De baseline gebruikt een naïeve sequentierepresentatie door alleen de **laatste timestep** van de (gepaddede) sequentie te gebruiken voor classificatie.  
Omdat sequenties **variabele lengte** hebben en op batch-niveau worden **gepaddede**, kan deze keuze informatieverlies veroorzaken of padding-effecten introduceren.

# Hypothese
Als we de sequentierepresentatie vervangen door **pooling over tijd** (bijv. gemiddelde over alle timesteps), dan wordt het model robuuster voor variabele sequentielengtes en verwachten we een **hogere en/of stabielere validatie-accuracy**.

# Experiment
We houden dataset, preprocessor, TrainerSettings en optimizer gelijk, en vergelijken:
- Baseline representatie: laatste timestep
- Pooling representatie: mean pooling over tijd

Resultaten worden gelogd in MLflow zodat runs direct vergelijkbaar zijn.


In [9]:
import sys
from pathlib import Path

# Vind repo-root door omhoog te lopen totdat we pyproject.toml of .git vinden
p = Path().resolve()
while not (p / "pyproject.toml").exists() and not (p / ".git").exists():
    if p.parent == p:
        raise RuntimeError("Repo root not found (no pyproject.toml/.git).")
    p = p.parent

PROJECT_ROOT = p
sys.path.insert(0, str(PROJECT_ROOT))

print("Project root added to sys.path:", PROJECT_ROOT)


Project root added to sys.path: /home/mischatrader1/MADS/MADS-MachineLearning-course


In [10]:
from pathlib import Path

import torch

from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

In [11]:
import torch

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Override naar CPU voor kleine RNN-modellen indien gewenst
device = torch.device("cpu")

device


Using CPU


device(type='cpu')

In [12]:
# Data + preprocessor (identiek aan notebook 01)
preprocessor = PaddedPreprocessor()

gestures_factory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gestures_factory.create_datastreamer(batchsize=32, preprocessor=preprocessor)

train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

len(train), len(valid)

2026-01-08 08:33:55.751 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/mischatrader1/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 2017.13it/s]


(81, 20)

In [13]:
accuracy = Accuracy()

settings = TrainerSettings(
    epochs=30,  # bewust korter dan "100": dit notebook is een gerichte vergelijking
    metrics=[accuracy],
    logdir=Path("gestures_ex3"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs={
        "save": False,
        "verbose": True,
        "patience": 5,
        "delta": 0.0,
    },
)
settings

epochs: 30
metrics: [Accuracy]
logdir: gestures_ex3
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [14]:
from src_rnn.models import RNNConfig
from src_rnn.run_experiment import run_experiment

In [16]:
# Run 1: Baseline representatie (laatste timestep)
config = RNNConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)

_ = run_experiment(
    model_name="gru_last",                 # baseline: last-step representatie
    config=config,
    settings=settings,
    trainstreamer=train.stream(),
    validstreamer=valid.stream(),
    device=device,
    experiment_name="gestures-ex3",
    run_name="02_baseline_gru_last_h64_l1",
)


2026-01-08 08:35:03.040 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures_ex3/20260108-083503


TypeError: run_experiment.<locals>.optimizer_factory() got an unexpected keyword argument 'lr'

In [15]:
# Run 2: Pooling fix (mean pooling over tijd)
config = RNNConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)

_ = run_experiment(
    model_name="gru_mean",                 # fix: mean-pooling representatie
    config=config,
    settings=settings,
    trainstreamer=train.stream(),
    validstreamer=valid.stream(),
    device=device,
    experiment_name="gestures-ex3",
    run_name="02_fix_gru_meanpool_h64_l1",
)

2026/01/08 08:34:11 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/08 08:34:11 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

TypeError: run_experiment.<locals>.optimizer_factory() got an unexpected keyword argument 'lr'